In [1]:
from QMLE_scipy import *
from explore import *

### 1. Test k=2

In [2]:
# set up initial guess
sigma = 1
lam = 0.5
gamma = 0.1
rho = 0.2
beta = np.array([1,2])
params = [sigma,lam,gamma, rho, *beta]
params

# set up input data
n = 4
T = 4
k = 2
alpha = np.random.normal(0,1,4)
c0 = np.random.normal(0,1,4)
x1 = []
for i in range(T):
    x1.append(np.random.normal(0,1,n*k).reshape(n,k))
y1 = []
for i in range(T+1):
    y1.append(np.random.normal(0,1,4))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = np.array([[0,1,0,0],[1,0,0,0],[0,0,0,1],[0,1,0,0]]).reshape(4,4)
w1 = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[1,0,0,0]]).reshape(4,4)
W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

In [3]:
# run optimization with constraints
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params,est_coeff = False)

/Users/helangchun/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [4]:
result.params

[0.5985128708323901,
 0.39547183896204746,
 0.7199917965804363,
 0.8419632110515725,
 array([ 0.11787892, -0.94836582])]

In [5]:
result.gamma()

0.7199917965804363

In [6]:
result.residual_mean()

-0.07903003992271959

In [7]:
result.residual_std()

0.6222983690459347

In [8]:
result.asymptotic_var()

TypeError: 'numpy.ndarray' object is not callable

In [8]:
result.print_table()

Residuals:
    Min         1Q    Median        3Q      Max
-------  ---------  --------  --------  -------
-1.7039  -0.435898  0.143706  0.838157  1.33128

Estimated Parameters:
   Sigma    Lambda     Gamma        Rho  Beta
--------  --------  --------  ---------  -------------------------
0.667192  0.312994  -0.11039  0.0288835  [ 0.23643699 -0.40589926]

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
0.0721547   0.499961  -0.236105  -0.336011

Estimated c0 (display first 10 rows): 
     c0_0      c0_1      c0_2      c0_3
---------  --------  --------  --------
-0.562749  0.771059  0.151581  0.434114

Estimated asympotitic variance: 


TypeError: 'int' object is not subscriptable

In [9]:
result = QMLE_scipy_estimate(x1, y1, W_ls,initial_guess = params, constrain= False)

In [10]:
result.print_table()

Residuals:
     Min         1Q      Median        3Q      Max
--------  ---------  ----------  --------  -------
-1.42818  -0.618803  -0.0494101  0.434941  1.98379

Estimated Parameters:
   Sigma     Lambda     Gamma      Rho  Beta
--------  ---------  --------  -------  -------------------------
0.515758  -0.610867  0.506756  0.25335  [-0.53539343  0.06759927]

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3
---------  ---------  ---------  ---------
 0.028828  -0.359024  -0.362082   0.692278

Estimated c0 (display first 10 rows): 
     c0_0       c0_1      c0_2       c0_3
---------  ---------  --------  ---------
-0.642871  -0.179963  0.610228  0.0628039

Estimated asympotitic variance: 
              Gamma         Rho      Beta_0      Beta_1      Lambda     Sigma^2
-------  ----------  ----------  ----------  ----------  ----------  ----------
Gamma     0.690924    0.922535    0.0550924   0.710401   -0.886098   -0.277231
Rho       0.922535    1.2

### 2. Test k=1

In [11]:
# set up initial guess
initial_guess = [1.5,0.2,0.5,0.3,1.2] # in the order sigma, lam, gamma, rho, beta (all real numbers)

n = 20
k = 1
T = 10

# generate samples
t_theta = [0.1,0.2,np.array([1]).reshape(k,1),0.5,1] # true parameters
t_gamma, t_rho, t_beta, t_lam, t_sig = t_theta
alpha = np.random.normal(0,1,T).reshape(T,1)
c0 = np.random.normal(0,1,n).reshape(n,1)
l_n = np.ones(n).reshape(n,1)
x = []
for i in range(T):
    x.append(np.random.normal(0,1,n).reshape(n,k))
# manually setting up a row-normalized spatial weight vector with 0 diagonals for even time points
w0 = generate_weight_matrix(n)
w1 = generate_weight_matrix(n)

W_ls = []
for i in range(T+1):
    if is_even(i):
        W_ls.append(w0)
    else:
        W_ls.append(w1)

        V_nt = []
for i in range(T):
    V_nt.append(np.random.normal(0,t_sig,n).reshape(n,1))

Y0 = np.random.normal(0,1,n).reshape(n,1)

y = []
y.append(Y0)
for i in range(T):
    c_vec = t_gamma*y[i]+t_rho*np.matmul(W_ls[i],np.array(y[i]).reshape(n,1))+np.matmul(x[i],np.array(t_beta).reshape(k,1))+c0+alpha[i][0]*l_n+V_nt[i]
    Y_nt = np.matmul(np.linalg.inv(np.identity(n)-t_lam*W_ls[i+1]),c_vec).reshape(n,1)
    y.append(Y_nt)

In [12]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [13]:
result.print_table()

Residuals:
     Min         1Q     Median       3Q      Max
--------  ---------  ---------  -------  -------
-2.96032  -0.887209  -0.177504  0.66945  2.53825

Estimated Parameters:
   Sigma    Lambda      Gamma       Rho    Beta
--------  --------  ---------  --------  ------
0.886987  0.586125  0.0841225  0.247246  1.0118

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3    alpha_4    alpha_5    alpha_6    alpha_7    alpha_8    alpha_9
---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
 0.635878   -0.62355  -0.953056  -0.848211  -0.790451   0.398596   0.844455   0.401626   0.749717   0.184994

Estimated c0 (display first 10 rows): 
     c0_0    c0_1       c0_2       c0_3      c0_4      c0_5     c0_6      c0_7      c0_8      c0_9
---------  ------  ---------  ---------  --------  --------  -------  --------  --------  --------
-0.665242  0.6764  -0.382798  -0.293082  0.756522  0.374974  1.0672

In [14]:
result = QMLE_scipy_estimate(x, y, W_ls,initial_guess = initial_guess, constrain= False)

In [15]:
result.print_table()

Residuals:
     Min         1Q     Median       3Q      Max
--------  ---------  ---------  -------  -------
-2.96032  -0.887209  -0.177504  0.66945  2.53825

Estimated Parameters:
   Sigma    Lambda      Gamma       Rho    Beta
--------  --------  ---------  --------  ------
0.886987  0.586125  0.0841225  0.247246  1.0118

Estimated alpha (display first 10 rows): 
  alpha_0    alpha_1    alpha_2    alpha_3    alpha_4    alpha_5    alpha_6    alpha_7    alpha_8    alpha_9
---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------  ---------
 0.635878   -0.62355  -0.953056  -0.848211  -0.790451   0.398596   0.844455   0.401626   0.749717   0.184994

Estimated c0 (display first 10 rows): 
     c0_0    c0_1       c0_2       c0_3      c0_4      c0_5     c0_6      c0_7      c0_8      c0_9
---------  ------  ---------  ---------  --------  --------  -------  --------  --------  --------
-0.665242  0.6764  -0.382798  -0.293082  0.756522  0.374974  1.0672